In [ ]:
# Silence wanring about recompiling tensorflow to use more optimal CPU instructions
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

## 2.1 A first look at a neural network
Creating, training, and evaluation a simple NN on MNIST

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
train, test = mnist.load_data()

In [ ]:
train_images, train_labels = train
test_images, test_labels = test

print(train_images.shape)
print(test_images.shape)
print(np.unique(train_labels))
print(train_images.dtype)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential([
    layers.Dense(2**9, activation='relu'),
    layers.Dense(10, activation='softmax'),
])

In [ ]:
model.compile(optimizer = 'rmsprop',
              loss      = 'sparse_categorical_crossentropy',
              metrics   = ['accuracy'])

In [ ]:
UINT8_MAX = 2**8 - 1
def prepare_images(t):
    return t.reshape((-1,28*28)).astype('float32') / UINT8_MAX

if train_images.ndim == 3:
    print(train_images.shape)
    train_images = prepare_images(train_images)
    test_images = prepare_images(test_images)
print(train_images.shape, test_images.shape)
print()

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=2**7)

In [ ]:
idx = 0
true = test_labels[idx]
pred_probs = model.predict(test_images[idx:idx+1])
pred = pred_probs.argmax()
prob = pred_probs[0,pred]
print(f'True = {true}; Pred = {pred} ({prob:.5%})')

In [ ]:
results = model.evaluate(test_images, test_labels., verbose=0)

In [ ]:
loss, acc = results
print(f'Loss = {loss}')
print(f'Acc = {acc}')

## 2.5 Build the NN from scratch

In [ ]:
import math

In [ ]:
# Dense layer: __init__, __call__
class NaiveDense():
    def __init__(self, units, activation=None):
        self.activation = activation
        self.units  = units
        self.w = None
        self.b = None

    def __call__(self, inputs):
        if self.w is None or self.b is None:
            w_shape = (inputs.shape[-1], self.units)
            self.w = tf.random.uniform(w_shape, 0, 1e-1)
            self.w = tf.Variable(self.w)

            b_shape = (self.units,)
            self.b = tf.zeros(b_shape)
            self.b = tf.Variable(self.b)
        return self.activation(inputs @ self.w + self.b)

    @property
    def weights(self):
        return [self.w, self.b]

class NaiveSequential():
    def __init__(self, layers):
        self.layers = layers
        self.learning_rate = 1e-3

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

    def fit(self, inputs, labels, epochs, batch_size):
        for epoch in range(epochs):
            print(f'Start of Epoch {epoch}')
            for x, y_true in batches(inputs, labels, batch_size):
                loss = self.update_weights(x, y_true)
            print(f'End of Epoch {epoch}; loss = {loss:.3f}')
    
    def update_weights(self, x, y_true):
        with tf.GradientTape() as tape:
            y_pred = self(x)
            #print(f'DEBUG :: 2) y_pred.shape={y_pred.shape}; y_true.dtype={y_true.shape}')
            loss = tf.losses.sparse_categorical_crossentropy(y_true, y_pred)
            avg_loss = tf.reduce_mean(loss)
        gradients = tape.gradient(avg_loss, self.weights)
        
        for g, w in zip(gradients, self.weights):
            w.assign_sub(g * self.learning_rate)
        
        return avg_loss

def batches(inputs, labels, batch_size):
    start = 0
    while start < len(inputs):
        stop = min(len(inputs), start + batch_size)
        yield inputs[start:stop], labels[start:stop]
        start += batch_size

In [ ]:
model = NaiveSequential([ 
    NaiveDense(2**9, activation=tf.nn.relu),
    NaiveDense(10, activation=tf.nn.softmax)
])

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=2**7)

In [ ]:
y_pred = model(test_images).numpy()
acc = (y_pred.argmax(axis=1) == test_labels).mean()
print(f'Accuracy : {acc:.0%}')

## 3.5.4 Building a linear classifier

In [ ]:
learning_rate = 0.1
n_samples = 1000

x_train1 = np.random.multivariate_normal(mean=[3,0], cov=[[1,0.5],[0.5,1]], size=n_samples).astype(np.float32)
x_train2 = np.random.multivariate_normal(mean=[0,3], cov=[[1,0.5],[0.5,1]], size=n_samples).astype(np.float32)
x_train = tf.concat([x_train1, x_train2], axis=0)
y_train = tf.concat([tf.ones((n_samples,1)), tf.zeros((n_samples,1))], axis=0)

input_dim = x_train.shape[-1]
output_dim = y_train.shape[-1]
w_init = tf.random.uniform(shape=(input_dim, output_dim))
b_init = tf.random.uniform(shape=(output_dim,))
w = tf.Variable(initial_value=w_init, name='slope')
b = tf.Variable(initial_value=b_init, name='bias')
trainable_weights = [w, b]

def model(x: tf.Tensor) -> tf.Tensor:
    return x @ w + b

def compute_loss(y_pred: tf.Tensor, y_true: tf.Tensor) -> float:
    return tf.reduce_mean(tf.square(y_pred - y_true))

def train_step(x: tf.Tensor, y_true: tf.Tensor) -> float:
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = compute_loss(y_pred, y_true)
    grads = tape.gradient(loss, trainable_weights)
    for grad, weight in zip(grads, trainable_weights):
        weight.assign_sub(grad * learning_rate)
    
    return loss

def plot(x_train, y_train, w, b):
    x = x_train[:,0]
    y = x_train[:,1]
    c = y_train[:,0]
    plt.scatter(x, y, c=c)

    slope = -w[0]/w[1]
    intercept = (0.5-b)/w[1]
    x_line = np.linspace(tf.reduce_min(x), tf.reduce_max(x))
    y_line = slope * x_line + intercept
    plt.plot(x_line, y_line, "-r")

plot(x_train, y_train, w, b)

In [ ]:
n_train_steps = 40
loss_init = None
for s in range(n_train_steps):
    loss = train_step(x_train, y_train)
    if loss_init is None:
        loss_init = loss
print(f'Loss : {loss_init} -> {loss}')
plot(x_train, y_train, w, b)

## 3.6.1  

## Model/Layer subclass

## Other